<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_Preprocessing/blob/main/AI/%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 51.1 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
from sklearn.datasets import load_files
from bs4 import BeautifulSoup
import re
import konlpy
import pandas as pd
import nltk
import tensorflow as tf
tf.random.set_seed(777) #하이퍼파라미터 튜닝을 위해 실행시 마다 변수가 같은 초기값 가지게 하기
from sklearn.model_selection import train_test_split
import numpy as np
import sys

sys.getdefaultencoding()

def clean_korean_documents(documents):
    #텍스트 정제 (HTML 태그 제거)
    for i, document in enumerate(documents):
        document = BeautifulSoup(document, 'html.parser').text 
        documents[i] = document

    #텍스트 정제 (특수기호 제거)
    for i, document in enumerate(documents):
        document = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', document) #특수기호 제거, 정규 표현식
        documents[i] = document

    #텍스트 정제 (형태소 분석)
    for i, document in enumerate(documents):
        okt = konlpy.tag.Okt()
        clean_words = []
        for word in okt.pos(document, stem=True): #어간 추출
            if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
                clean_words.append(word[0])
        document = ' '.join(clean_words)
        documents[i] = document

    #텍스트 정제 (불용어 제거)
    df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
    df[0] = df[0].apply(lambda x: x.strip())
    stopwords = df[0].to_numpy()
    nltk.download('punkt')
    for i, document in enumerate(documents):
        clean_words = [] 
        for word in nltk.tokenize.word_tokenize(document): 
            if word not in stopwords: #불용어 제거
                clean_words.append(word)
        documents[i] = ' '.join(clean_words)  

    return documents


In [ ]:
##########데이터 로드

naver_news = load_files('/gdrive/My Drive/Colab Notebooks/Src/newsData/newsData', shuffle=True)
labels = ['정치', '경제', '사회', '생활/문화', '세계', '기술/IT', '연예', '스포츠']

# naver_news = load_files('/gdrive/My Drive/Colab Notebooks/Src/QueryData', shuffle=True, encoding='CP949')
# labels = []
# for i in range(0, 20):
#   labels.append(str(i))

##########데이터 분석

##########데이터 전처리

x_data = naver_news.data
y_data = naver_news.target
# print(len(x_data)) #1600
x_data = x_data[:800] #데이터를 800개로 제한
y_data = y_data[:800]

print(x_data[0]) #b'\xed\x83\x9c\xec\x96\x91\xea\xb4\x91\xec\x9c\xbc\xeb\xa1\x9c \xec\xb6\xa9\xec\xa0\x84\xeb\x90\x98\xeb\x8a\x94 \xe2\x80\x98\xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0X ...'
print(y_data[0]) #5

x_data = [x.decode('utf-8') for x in x_data] #바이트를 문자열로 바꾸기
print(x_data[0]) #태양광으로 충전되는 ‘아이폰X 테슬라’ (지디넷코리아=이정현 기자)도널드 트럼프 미국 대통령과 푸틴 러시아 대통령의 초상화를 새긴 황금 아이폰을 출시해 ...

x_data = clean_korean_documents(x_data) #텍스트 정제
tokenizer = tf.keras.preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(x_data)
x_data = tokenizer.texts_to_sequences(x_data) #정수 인코딩
sequence_length = 200
x_data = tf.keras.preprocessing.sequence.pad_sequences(x_data, maxlen=sequence_length) #길이 맞추기

y_data = tf.keras.utils.to_categorical(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777, stratify=y_data)


##########모델 생성

input = tf.keras.layers.Input(shape=(sequence_length,))
net = tf.keras.layers.Dense(units=32, activation='relu')(input)
net = tf.keras.layers.Dense(units=32, activation='relu')(net)
net = tf.keras.layers.Dense(units=8, activation='softmax')(net)
model = tf.keras.models.Model(input, net)


b'\xed\x83\x9c\xec\x96\x91\xea\xb4\x91\xec\x9c\xbc\xeb\xa1\x9c \xec\xb6\xa9\xec\xa0\x84\xeb\x90\x98\xeb\x8a\x94 \xe2\x80\x98\xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0X \xed\x85\x8c\xec\x8a\xac\xeb\x9d\xbc\xe2\x80\x99\t(\xec\xa7\x80\xeb\x94\x94\xeb\x84\xb7\xec\xbd\x94\xeb\xa6\xac\xec\x95\x84=\xec\x9d\xb4\xec\xa0\x95\xed\x98\x84 \xea\xb8\xb0\xec\x9e\x90)\xeb\x8f\x84\xeb\x84\x90\xeb\x93\x9c \xed\x8a\xb8\xeb\x9f\xbc\xed\x94\x84 \xeb\xaf\xb8\xea\xb5\xad \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xea\xb3\xbc \xed\x91\xb8\xed\x8b\xb4 \xeb\x9f\xac\xec\x8b\x9c\xec\x95\x84 \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xec\x9d\x98 \xec\xb4\x88\xec\x83\x81\xed\x99\x94\xeb\xa5\xbc \xec\x83\x88\xea\xb8\xb4 \xed\x99\xa9\xea\xb8\x88 \xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0\xec\x9d\x84 \xec\xb6\x9c\xec\x8b\x9c\xed\x95\xb4 \xed\x99\x94\xec\xa0\x9c\xeb\xa5\xbc \xeb\xaa\xa8\xec\x95\x98\xeb\x8d\x98 \xeb\x9f\xac\xec\x8b\x9c\xec\x95\x84 \xec\x97\x85\xec\xb2\xb4 \xec\xba\x90\xeb\xb9\x84\xec\x95\x84(Caviar)\xea\xb0\x80 \xec\x9d\xb4\xe

In [ ]:
##########모델 학습

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test)) 

##########모델 검증

##########모델 예측

x_test = np.array([
     '우리은행, 인공지능 기반 시장예측시스템 구축 추진'
])
x_test = clean_korean_documents(x_test) #텍스트 정제
x_test = tokenizer.texts_to_sequences(x_test) #정수 인코딩
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=sequence_length) #길이 맞추기

y_predict = model.predict(x_test)

label = labels[y_predict[0].argmax()]
confidence = y_predict[0][y_predict[0].argmax()]
print(label, confidence) #

Epoch 1/50
18/18 [==============================] - 1s 12ms/step - loss: 2329.0676 - accuracy: 0.1393 - val_loss: 1713.7452 - val_accuracy: 0.1125
Epoch 2/50
18/18 [==============================] - 0s 4ms/step - loss: 1032.8552 - accuracy: 0.1446 - val_loss: 1042.3944 - val_accuracy: 0.1250
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 632.8298 - accuracy: 0.1696 - val_loss: 777.6588 - val_accuracy: 0.1125
Epoch 4/50
18/18 [==============================] - 0s 3ms/step - loss: 453.0080 - accuracy: 0.1875 - val_loss: 657.7955 - val_accuracy: 0.1208
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 337.1630 - accuracy: 0.2482 - val_loss: 572.7471 - val_accuracy: 0.1125
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 260.0916 - accuracy: 0.2786 - val_loss: 521.3692 - val_accuracy: 0.0917
Epoch 7/50
18/18 [==============================] - 0s 3ms/step - loss: 201.6629 - accuracy: 0.3214 - val_loss: 477.5094 - val_accu